In [8]:
import kagglehub
irkaal_foodcom_recipes_and_reviews_path = kagglehub.dataset_download('irkaal/foodcom-recipes-and-reviews')
print('Data source import complete.')

Data source import complete.


In [27]:
!pip install pyreclab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.2/225.2 kB 3.6 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split
import pyreclab

In [2]:
recipes=pd.read_csv('/kaggle/input/foodcom-recipes-and-reviews/recipes.csv')

In [3]:
recipes.drop(["RecipeServings", "RecipeYield", "AggregatedRating", "ReviewCount"], inplace=True, axis=1)
recipes.fillna("-", inplace=True)

In [4]:
recipes_util = recipes.copy()
columns = ['RecipeId', 'Name', 'RecipeCategory','RecipeIngredientParts','Calories','FatContent','SaturatedFatContent','CholesterolContent','SodiumContent','CarbohydrateContent','FiberContent','SugarContent','ProteinContent']
recipes_util = recipes_util[columns]

In [5]:
#Valores nutricionales "sanos" y restricciones alimentarias, pueden ser adaptables según el usuario
#https://www-verywellfit-com.translate.goog/what-does-based-on-a-2000-calorie-diet-mean-4099137?_x_tr_sl=en&_x_tr_tl=es&_x_tr_hl=es&_x_tr_pto=sge#:~:text=Based%20on%20the%20DV%20information,or%201200%20calories%20from%20carbohydrates

Daily_values = {"Calories": 2000,
                "FatContent": 78,
                "SaturatedFatContent": 20,
                "CholesterolContent": 300,
                "SodiumContent": 2300,
                "CarbohydrateContent": 275,
                "FiberContent": 28,
                "SugarContent": 40,
                "ProteinContent": 50}

Meal_values = Daily_values.copy()
for value in Meal_values:
    Meal_values[value] = Meal_values[value]/3 #Son 3 comidas al día

Restrictions = ["cream", "milk", "yogurt", "cheese"] #Intolerancia a la lactosa

In [6]:
for key, value in Meal_values.items():
    recipes_util = recipes_util[recipes_util[key] <= value]

for restriction in Restrictions:
    recipes_util = recipes_util[~recipes_util['RecipeIngredientParts'].str.contains(restriction, case=False, regex=False)]

**Most Popular**

In [7]:
reviews = pd.read_csv('/kaggle/input/foodcom-recipes-and-reviews/reviews.csv')

In [8]:
reviews_util = reviews[["ReviewId", "RecipeId", "Rating"]]
reviews_util = reviews_util[reviews_util['RecipeId'].isin(recipes_util['RecipeId'])]

In [9]:
train, val = train_test_split(reviews_util, test_size=0.2, random_state=17)

train.to_csv("train.csv", index=False, header=False)
val.to_csv("val.csv", index=False, header=False)

In [10]:
mp = pyreclab.MostPopular(dataset="train.csv", dlmchar=b',', usercol=0, itemcol=1, ratingcol=2)
mp.train(progress=True)

In [ ]:
recommendationList, map, ndcg = mp.testrec(input_file = "val.csv",
                                                 dlmchar = b',',
                                                 header = False,
                                                 usercol = 0,
                                                 itemcol = 1,
                                                 ratingcol = 2,
                                                 topn = 10,
                                                 output_file = 'ranking.json',
                                                 relevance_threshold = 2,
                                                 includeRated = False )

In [19]:
recipe_id = 50
ranking = [int(r) for r in mp.recommend(str(recipe_id), 10, includeRated=False)]

In [22]:
valid_ranking = [r for r in ranking if r in recipes_util['RecipeId']]
recipes_util.iloc[valid_ranking]

,RecipeId,Name,RecipeCategory,RecipeIngredientParts,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent
206336,215257,Athen's Marinated Black-Eyed Peas,Vegetable,"c(""black-eyed peas"", ""onion"", ""green bell pepp...",377.6,22.8,3.1,0.0,743.4,35.8,6.1,11.1,8.4
